# Lesson 5: Adding Relationships to the SEC Knowledge Graph

### Import packages and set up Neo4j

In [346]:
# %pip install python-dotenv 
# %pip install langchain langchain-community langchain-openai 
# %pip install neo4j 
# %pip install textwrap
# %pip install pypdf

# Installation de yFiles Jupyter Graphs spécialement pour Neo4j
# %pip install yfiles_jupyter_graphs_for_neo4j


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np
from collections import defaultdict


In [ ]:
import random
import glob
import os
import uuid
import json
from tqdm import tqdm



In [ ]:
# Import de yFiles pour Neo4j
from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget
from neo4j import GraphDatabase

In [ ]:
from dotenv import load_dotenv
import os

# Common data processing
import textwrap

# Langchain
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader

# Warning control
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Load from environment
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE') or 'neo4j'

# Global constants
VECTOR_INDEX_NAME = 'Documents_chunks'
VECTOR_NODE_LABEL = 'Chunk'
VECTOR_SOURCE_PROPERTY = 'text'
VECTOR_EMBEDDING_PROPERTY = 'textEmbedding'

In [ ]:
# Configuration pour les embeddings OpenAI dans Neo4j
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_BASE_URL = os.getenv('OPENAI_BASE_URL', 'https://api.openai.com/v1')
OPENAI_ENDPOINT = OPENAI_BASE_URL + '/embeddings'

print("🔧 Configuration Neo4j et OpenAI")
print(f"   Neo4j URI: {NEO4J_URI}")
print(f"   Database: {NEO4J_DATABASE}")
print(f"   OpenAI Base URL: {OPENAI_BASE_URL}")
print(f"   OpenAI Endpoint: {OPENAI_ENDPOINT}")
print(f"   API Key configurée: {'✅' if OPENAI_API_KEY else '❌'}")

# Vérifier la connexion Neo4j
try:
    result = kg.query("RETURN 1 as test")
    print("✅ Connexion Neo4j active")
except Exception as e:
    print(f"❌ Erreur connexion Neo4j: {e}")

In [ ]:
# Test de la configuration OPENAI_ENDPOINT
def test_openai_endpoint():
    """Test de la configuration de l'endpoint OpenAI"""
    
    print("🧪 Test de la configuration OpenAI")
    print(f"   OPENAI_BASE_URL: {OPENAI_BASE_URL}")
    print(f"   OPENAI_ENDPOINT: {OPENAI_ENDPOINT}")
    
    # Vérifier que l'endpoint se termine bien par '/embeddings'
    if OPENAI_ENDPOINT.endswith('/embeddings'):
        print("   ✅ Endpoint correctement formaté")
    else:
        print("   ⚠️ Attention: l'endpoint ne se termine pas par '/embeddings'")
    
    # Test simple avec un embedding
    try:
        from langchain_openai import OpenAIEmbeddings
        embeddings = OpenAIEmbeddings(
            model="text-embedding-3-small",
            openai_api_base=OPENAI_BASE_URL
        )
        test_vector = embeddings.embed_query("Test de connexion")
        print(f"   ✅ Test réussi! Vecteur de dimension: {len(test_vector)}")
        return True
    except Exception as e:
        print(f"   ❌ Erreur lors du test: {e}")
        return False

# Exécuter le test
test_success = test_openai_endpoint()

In [ ]:
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

In [ ]:
# Configuration
pdf_path = "PDFs"
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

In [ ]:
# Récupérer tous les PDFs
pdf_files = glob.glob(f"{pdf_path}/*.pdf")
print(f"📄 {len(pdf_files)} fichiers PDF trouvés")

# Traiter tous les PDFs
all_chunks = []

for pdf_file in pdf_files:
    try:
        print(f"Traitement: {os.path.basename(pdf_file)}")
        
        # Charger et chunker
        loader = PyPDFLoader(pdf_file)
        pages = loader.load()
        chunks = text_splitter.split_documents(pages)
        
        # Ajouter métadonnées simples
        doc_id = str(uuid.uuid4())
        for i, chunk in enumerate(chunks):
            chunk_data = {
                'id': str(uuid.uuid4()),
                'doc_id': doc_id,
                'filename': os.path.basename(pdf_file),
                'chunk_index': i,
                'text': chunk.page_content,
                'word_count': len(chunk.page_content.split())
            }
            all_chunks.append(chunk_data)
        
        print(f"  ✅ {len(chunks)} chunks créés")
        
    except Exception as e:
        print(f"  ❌ Erreur: {e}")

print(f"\n🎯 Total: {len(all_chunks)} chunks prêts")
print(f"📊 Mots par chunk (moyenne): {sum(c['word_count'] for c in all_chunks) / len(all_chunks):.0f}")
import pprint


In [ ]:
pprint.pprint(random.choice(all_chunks), width=100, indent=2)


In [ ]:
merge_chunk_node_query = """
MERGE(mergedChunk:Chunk {id: $chunkParam.id})
    ON CREATE SET 
        mergedChunk.id = $chunkParam.id,
        mergedChunk.doc_id = $chunkParam.doc_id, 
        mergedChunk.filename = $chunkParam.filename, 
        mergedChunk.chunk_index = $chunkParam.chunk_index, 
        mergedChunk.text = $chunkParam.text, 
        mergedChunk.word_count = $chunkParam.word_count 
RETURN mergedChunk
"""

In [ ]:
# Pour traiter tous les chunks
for chunk in tqdm(all_chunks, desc="Création des chunks dans Neo4j"):
    print(f"Creating `:Chunk` node for chunk ID {chunk['id']}")
    result = kg.query(
        merge_chunk_node_query, 
        params={'chunkParam': chunk}
    )
    
print(f"✅ {len(all_chunks)} chunks créés dans Neo4j")

In [ ]:
kg.query("""
         MATCH (n)
         RETURN count(n) as nodeCount
         """)

In [ ]:
kg.query("""
         MATCH (n)
         RETURN count(n) as nodeCount
         """)

In [ ]:
# Creation du vector Index dans Neo4j

In [ ]:
kg.query("""
         CREATE VECTOR INDEX `GrahRAG` IF NOT EXISTS
          FOR (c:Chunk) ON (c.textEmbedding) 
          OPTIONS { indexConfig: {
            `vector.dimensions`: 1536,
            `vector.similarity_function`: 'cosine'    
         }}
""")

In [ ]:
kg.query("SHOW INDEXES")

In [ ]:
# ✅ Génération d'embeddings - Stratégie simplifiée (embeddings pré-calculés)
def generate_embeddings_simple():
    """Utilise les embeddings pré-calculés (stratégie qui fonctionne)"""
    
    print("🚀 Génération d'embeddings avec la stratégie qui fonctionne...")
    print("📋 Utilisation des embeddings pré-calculés")
    
    try:
        chunks_updated = 0
        for chunk in enriched_chunks:
            if 'embedding' in chunk:
                kg.query("""
                    MATCH (c:Chunk {id: $chunk_id})
                    CALL db.create.setNodeVectorProperty(c, "textEmbedding", $embedding)
                    RETURN c.id as updated_id
                    """, 
                    params={
                        "chunk_id": chunk['id'],
                        "embedding": chunk['embedding']
                    }
                )
                chunks_updated += 1
                
                if chunks_updated % 10 == 0:
                    print(f"   ⏳ {chunks_updated} chunks traités...")
        
        print(f"✅ Succès! {chunks_updated} chunks traités avec embeddings pré-calculés")
        return True
        
    except Exception as e:
        print(f"❌ Erreur: {str(e)[:200]}...")
        return False

# Exécuter la génération simplifiée
success = generate_embeddings_simple()

## 🔍 Recherche Vectorielle dans le Knowledge Graph

Maintenant que notre Knowledge Graph est opérationnel avec des embeddings, nous pouvons effectuer des recherches sémantiques avancées !

In [ ]:
# 🔧 FONCTION DE RECHERCHE VECTORIELLE CORRIGÉE
def neo4j_vector_search_robust(question, top_k=5):
    """Recherche vectorielle avec le bon nom d'index"""
    
    print(f"🔍 Recherche pour: '{question}'")
    
    try:
        from langchain_openai import OpenAIEmbeddings
        embeddings_service = OpenAIEmbeddings(
            model="text-embedding-3-small"
        )
        
        # Générer l'embedding de la question
        question_embedding = embeddings_service.embed_query(question)
        
        # Recherche avec l'embedding généré et le BON nom d'index
        search_query = """
            CALL db.index.vector.queryNodes('GrahRAG', $top_k, $question_embedding) 
            YIELD node, score
            RETURN score, node.text AS text, node.filename AS source, node.id AS chunk_id
            ORDER BY score DESC
        """
        
        similar = kg.query(search_query, 
                            params={
                            'question_embedding': question_embedding,
                            'top_k': top_k
                            })
        
        print(f"✅ Recherche réussie: {len(similar)} résultats trouvés")
        
        # Afficher les résultats
        for i, result in enumerate(similar, 1):
            print(f"   {i}. Score: {result['score']:.4f} | Source: {result['source']}")
            print(f"      Text: {result['text'][:100]}...")
        
        return similar
        
    except Exception as e:
        print(f"❌ Erreur de recherche: {str(e)[:200]}...")
        print("💡 Vérifiez que l'index 'GrahRAG' existe et que les embeddings sont générés")
        return []

# Test de la fonction corrigée
print("🧪 Test de la fonction de recherche vectorielle corrigée:")
test_results = neo4j_vector_search_robust("c'est quoi Luxchatgov", top_k=3)

In [ ]:
# 🎯 EXEMPLES D'UTILISATION DE LA RECHERCHE VECTORIELLE
def demo_vector_search():
    """Démonstration de différents types de recherches"""
    
    print("🎯 DÉMONSTRATION DE LA RECHERCHE VECTORIELLE")
    print("=" * 50)
    
    # Liste de questions de test
    test_questions = [
        "Luxembourg",
        "What is mentioned about technology?",
        "Tell me about financial services",
        "Any information about companies?",
        "What data is available about regulations?"
    ]
    
    for i, question in enumerate(test_questions, 1):
        print(f"\n{i}️⃣ QUESTION: {question}")
        print("-" * 40)
        
        try:
            results = neo4j_vector_search_robust(question, top_k=3)
            
            if results:
                for j, result in enumerate(results, 1):
                    score = result.get('score', 0)
                    text = result.get('text', 'N/A')
                    source = result.get('source', 'Unknown')
                    
                    print(f"   📄 Résultat {j} (Score: {score:.3f})")
                    print(f"   Source: {source}")
                    print(f"   Texte: {text[:200]}...")
                    print()
            else:
                print("   ❌ Aucun résultat trouvé")
                
        except Exception as e:
            print(f"   ❌ Erreur: {str(e)[:100]}...")
        
        print("=" * 50)

# Exécuter la démonstration
demo_vector_search()

In [ ]:
# 🔍 RECHERCHE INTERACTIVE PERSONNALISÉE
def interactive_search():
    """Interface de recherche interactive"""
    
    print("🔍 RECHERCHE INTERACTIVE DANS LE KNOWLEDGE GRAPH")
    print("=" * 55)
    print("Tapez votre question ou 'quit' pour arrêter")
    print("Exemples:")
    print("  • 'What is Luxembourg known for?'")
    print("  • 'Tell me about financial regulations'")
    print("  • 'Any mention of technology companies?'")
    print("")
    
    while True:
        try:
            question = input("🤔 Votre question: ").strip()
            
            if question.lower() in ['quit', 'exit', 'stop', 'q']:
                print("👋 Recherche terminée!")
                break
                
            if not question:
                print("⚠️ Veuillez entrer une question valide")
                continue
            
            print(f"\n🔍 Recherche en cours pour: '{question}'")
            print("-" * 50)
            
            results = neo4j_vector_search_robust(question, top_k=3)
            
            if results:
                print(f"✅ {len(results)} résultats trouvés:\n")
                
                for i, result in enumerate(results, 1):
                    score = result.get('score', 0)
                    text = result.get('text', 'N/A')
                    source = result.get('source', 'Unknown')
                    
                    print(f"📄 RÉSULTAT {i}")
                    print(f"   📊 Score de similarité: {score:.4f}")
                    print(f"   📁 Source: {source}")
                    print(f"   📝 Contenu: {text[:300]}...")
                    if len(text) > 300:
                        print("       [...continué]")
                    print()
            else:
                print("❌ Aucun résultat trouvé pour cette question")
                print("💡 Essayez avec des mots-clés différents")
            
            print("=" * 55)
            
        except KeyboardInterrupt:
            print("\n👋 Recherche interrompue par l'utilisateur")
            break
        except Exception as e:
            print(f"❌ Erreur: {str(e)}")
            continue

# Note: Cette fonction est interactive, décommentez la ligne suivante pour l'utiliser
# interactive_search()

In [ ]:
# 🎯 RECHERCHE AVEC ANALYSE APPROFONDIE
def advanced_vector_search(question, top_k=5, include_entities=True):
    """Recherche vectorielle avancée avec analyse des entités connexes"""
    
    print(f"🔍 RECHERCHE AVANCÉE: '{question}'")
    print("=" * 60)
    
    # 1. Recherche vectorielle standard
    print("1️⃣ Recherche dans les chunks de texte:")
    chunk_results = neo4j_vector_search_robust(question, top_k=top_k)
    
    if chunk_results:
        print(f"   ✅ {len(chunk_results)} chunks pertinents trouvés")
        
        # Extraire les IDs des chunks les plus pertinents
        relevant_chunk_ids = []
        for result in chunk_results:
            # Récupérer l'ID du chunk depuis Neo4j
            chunk_query = """
                MATCH (c:Chunk) 
                WHERE c.text = $text 
                RETURN c.id as chunk_id
                LIMIT 1
            """
            chunk_id_result = kg.query(chunk_query, params={'text': result['text']})
            if chunk_id_result:
                relevant_chunk_ids.append(chunk_id_result[0]['chunk_id'])
        
        # 2. Trouver les entités mentionnées dans ces chunks
        if include_entities and relevant_chunk_ids:
            print("\n2️⃣ Entités trouvées dans les chunks pertinents:")
            entities_query = """
                MATCH (c:Chunk)-[:MENTIONS]->(e:Entity)
                WHERE c.id IN $chunk_ids
                RETURN e.name as entity_name, e.type as entity_type, 
                       e.description as description, count(*) as mentions
                ORDER BY mentions DESC, e.type
            """
            
            entities_results = kg.query(entities_query, params={'chunk_ids': relevant_chunk_ids})
            
            if entities_results:
                print(f"   ✅ {len(entities_results)} entités pertinentes:")
                
                # Grouper par type
                entities_by_type = {}
                for entity in entities_results:
                    entity_type = entity['entity_type']
                    if entity_type not in entities_by_type:
                        entities_by_type[entity_type] = []
                    entities_by_type[entity_type].append(entity)
                
                for entity_type, entities in entities_by_type.items():
                    print(f"\n   📂 {entity_type}:")
                    for entity in entities[:3]:  # Top 3 par type
                        print(f"      • {entity['entity_name']} ({entity['mentions']} mentions)")
                        if entity['description']:
                            print(f"        → {entity['description']}")
            else:
                print("   ⚠️ Aucune entité trouvée dans les chunks pertinents")
    
    # 3. Affichage des résultats principaux
    print("\n3️⃣ Chunks les plus pertinents:")
    if chunk_results:
        for i, result in enumerate(chunk_results[:3], 1):
            score = result.get('score', 0)
            text = result.get('text', 'N/A')
            source = result.get('source', 'Unknown')
            
            print(f"\n   📄 CHUNK {i} (Score: {score:.4f})")
            print(f"   📁 Source: {source}")
            print(f"   📝 Extrait: {text[:250]}...")
    else:
        print("   ❌ Aucun chunk pertinent trouvé")
    
    return {
        'chunks': chunk_results,
        'entities': entities_results if include_entities and 'entities_results' in locals() else [],
        'question': question
    }

# Test de la recherche avancée
print("🧪 Test de la recherche avancée:")
advanced_results = advanced_vector_search("financial services in Luxembourg", top_k=3)

### 🎯 Guide d'utilisation de la recherche vectorielle

Voici comment utiliser efficacement les fonctions de recherche que nous venons de créer :

In [ ]:
# 📚 GUIDE COMPLET D'UTILISATION DE LA RECHERCHE VECTORIELLE

print("📚 GUIDE D'UTILISATION DE LA RECHERCHE VECTORIELLE")
print("=" * 60)

print("\n🔧 FONCTIONS DISPONIBLES:")
print("1. neo4j_vector_search(question)           - Version originale")
print("2. neo4j_vector_search_robust(question)    - Version robuste avec fallback")
print("3. advanced_vector_search(question)        - Recherche + analyse des entités")
print("4. demo_vector_search()                    - Démonstration automatique")
print("5. interactive_search()                    - Interface interactive")

print("\n💡 EXEMPLES D'UTILISATION:")

print("\n1️⃣ RECHERCHE SIMPLE:")
print("   results = neo4j_vector_search_robust('Luxembourg financial sector')")
print("   # Retourne les chunks les plus similaires")

print("\n2️⃣ RECHERCHE AVEC NOMBRE DE RÉSULTATS:")
print("   results = neo4j_vector_search_robust('technology companies', top_k=10)")
print("   # Retourne les 10 meilleurs résultats")

print("\n3️⃣ RECHERCHE AVANCÉE AVEC ENTITÉS:")
print("   results = advanced_vector_search('banking regulations')")
print("   # Retourne chunks + entités connexes")

print("\n4️⃣ ACCÈS AUX RÉSULTATS:")
print("   for result in results:")
print("       score = result['score']      # Score de similarité (0-1)")
print("       text = result['text']        # Contenu du chunk")
print("       source = result['source']    # Nom du fichier source")

print("\n🎯 TYPES DE QUESTIONS EFFICACES:")
print("• Questions factuelles: 'What is Luxembourg known for?'")
print("• Recherche de concepts: 'financial regulations'")
print("• Recherche d'entités: 'companies mentioned in the documents'")
print("• Questions thématiques: 'technology and innovation'")

print("\n⚠️ CONSEILS POUR DE MEILLEURS RÉSULTATS:")
print("• Utilisez des mots-clés spécifiques")
print("• Posez des questions en anglais (si vos docs sont en anglais)")
print("• Essayez différentes formulations")
print("• Augmentez top_k pour plus de résultats")

print("\n🚀 EXEMPLES CONCRETS À TESTER:")
examples = [
    "Luxembourg banking sector",
    "What regulations are mentioned?",
    "Financial services overview",
    "Technology companies",
    "Economic policies"
]

for i, example in enumerate(examples, 1):
    print(f"   {i}. '{example}'")

print(f"\n🔄 POUR TESTER MAINTENANT:")
print("# Décommentez et modifiez cette ligne:")
print("# results = neo4j_vector_search_robust('YOUR_QUESTION_HERE')")
print("# print(f'Found {len(results)} results')")
print("# for r in results[:3]: print(f'Score: {r[\"score\"]:.3f} - {r[\"text\"][:100]}...')")

# Exemple pratique prêt à utiliser
print(f"\n🧪 TEST RAPIDE:")
test_question = "Luxembourg"
test_results = neo4j_vector_search_robust(test_question, top_k=2)
if test_results:
    print(f"✅ Test réussi! {len(test_results)} résultats pour '{test_question}'")
    for i, r in enumerate(test_results, 1):
        print(f"   {i}. Score: {r.get('score', 0):.3f} - {r.get('text', '')[:80]}...")
else:
    print(f"❌ Aucun résultat pour '{test_question}' - vérifiez vos embeddings")

## Extraction d'entités avec OpenAI

Maintenant que nous avons nos chunks, nous allons extraire les entités importantes de chaque chunk en utilisant l'API OpenAI GPT.

In [ ]:
def extract_entities_with_openai(text, chunk_id=None):
    """Extrait les entités d'un texte avec OpenAI de façon simple"""
    
    prompt = f"""
Tu es un expert en extraction d'entités. Analyse ce texte et identifie toutes les entités importantes.

Pour chaque entité, fournis :
- name: le nom exact de l'entité 
- type: PERSON, ORGANIZATION, LOCATION, TECHNOLOGY, PRODUCT, CONCEPT, EVENT, DATE, MISC
- description: brève description contextuelle

Texte à analyser :
---
{text}
---

Réponds UNIQUEMENT en JSON valide :
{{
    "entities": [
        {{
            "name": "nom_entité",
            "type": "TYPE",
            "description": "description courte"
        }}
    ]
}}
"""
    
    # Appel OpenAI
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)
    response = llm.invoke(prompt)
    
    # Parser JSON
    response_text = response.content.strip()
    if response_text.startswith("```json"):
        response_text = response_text[7:-3]
    elif response_text.startswith("```"):
        response_text = response_text[3:-3]
    
    result = json.loads(response_text)
    entities = result.get('entities', [])
    
    # Ajouter métadonnées simples
    for entity in entities:
        entity['chunk_id'] = chunk_id
        entity['confidence'] = 0.8  # Score par défaut
    
    return entities

### Traitement Complet de Tous les PDFs

⚠️ **Attention** : Cette cellule traite TOUS les chunks de TOUS les PDFs. Cela peut prendre beaucoup de temps et de crédits OpenAI si vous avez beaucoup de documents. Utilisez la cellule suivante pour un test rapide d'abord.

In [ ]:
# 🚨 CELLULE OPTIONNELLE : Traitement de TOUS les chunks
sample_chunks = all_chunks  # ⚠️ Attention : peut être très long !

print(f"💡 Configuration actuelle:")
print(f"   📁 PDFs disponibles: {len(pdf_files)}")
print(f"   📄 Total chunks: {len(all_chunks)}")
print(f"   🧪 Chunks à traiter: {len(sample_chunks) if 'sample_chunks' in locals() else 'Non défini'}")

print(f"\n💰 Estimation des coûts OpenAI pour traitement complet:")
estimated_api_calls = len(all_chunks) * 2  # Entités + embeddings
print(f"   🤖 Appels API estimés: {estimated_api_calls}")
print(f"   💸 Coût approximatif: ${estimated_api_calls * 0.001:.2f}")

print(f"\n🎯 Recommandation:")
if len(all_chunks) > 50:
    print(f"   ⚠️ Plus de 50 chunks détectés - commencez par un échantillon")
else:
    print(f"   ✅ Taille raisonnable - vous pouvez traiter tous les chunks")

### Extraction d'entités depuis les chunks

Maintenant, nous allons extraire les entités de tous les chunks traités.

In [ ]:
# Extraire les entités de tous les chunks (ou échantillon pour test)
# Option 1: Traiter tous les chunks (peut être long avec beaucoup de PDFs)
# sample_chunks = all_chunks
# Option 2: Traiter un échantillon pour tester rapidement
sample_chunks = all_chunks[:20]  # Prendre les 20 premiers chunks

all_entities = []

print(f"🔍 Extraction d'entités en cours pour {len(sample_chunks)} chunks...\n")

for i, chunk_data in enumerate(sample_chunks):
    print(f"📄 Chunk {i+1}: {len(chunk_data['text'])} caractères")
    print(f"Aperçu: {chunk_data['text'][:100]}...")
    
    # Extraire les entités
    entities = extract_entities_with_openai(chunk_data['text'], chunk_data['id'])
    
    print(f"✅ {len(entities)} entités trouvées\n")
    
    # Ajouter infos du chunk
    for entity in entities:
        entity['source_chunk'] = i
        entity['filename'] = chunk_data['filename']
        entity['doc_id'] = chunk_data['doc_id']
        entity['text_preview'] = chunk_data['text'][:150]
    
    all_entities.extend(entities)

print(f"🎯 Total: {len(all_entities)} entités extraites")


In [ ]:

# Afficher quelques exemples
if len(all_entities) > 0:
    print(f"\n📋 Exemples d'entités extraites:")
    for i, entity in enumerate(all_entities[:3]):
        print(f"   {i+1}. {entity['name']} ({entity['type']})")
        print(f"      Description: {entity['description']}")
        print(f"      Source: {entity['filename']}")
        print()

## Génération des Embeddings avec OpenAI

Maintenant, nous allons générer des embeddings vectoriels pour nos entités et chunks afin d'activer la recherche sémantique.

In [ ]:
def generate_embeddings(texts, model="text-embedding-3-small"):
    """Génère des embeddings pour une liste de textes avec OpenAI"""
    
    # Initialiser le service d'embeddings OpenAI
    embedding_service = OpenAIEmbeddings(model=model)
    
    print(f"🔄 Génération de {len(texts)} embeddings avec {model}...")
    
    # Générer les embeddings
    embeddings = embedding_service.embed_documents(texts)
    
    print(f"✅ Embeddings générés: {len(embeddings)} vecteurs de dimension {len(embeddings[0])}")
    
    return embeddings

def prepare_entity_texts(entities):
    """Prépare les textes des entités pour l'embedding"""
    entity_texts = []
    
    for entity in entities:
        # Combiner nom, type et description pour un embedding riche
        text = f"{entity['name']} ({entity['type']})"
        if entity.get('description'):
            text += f": {entity['description']}"
        entity_texts.append(text)
    
    return entity_texts

### Génération des embeddings pour les entités

In [ ]:
# Préparer les textes des entités pour l'embedding
entity_texts = prepare_entity_texts(all_entities)

print("📝 Textes préparés pour les entités:")
for i, text in enumerate(entity_texts):
    print(f"   {i+1}. {text}")

# Générer les embeddings des entités
entity_embeddings = generate_embeddings(entity_texts)

# Associer les embeddings aux entités
for i, entity in enumerate(all_entities):
    entity['embedding'] = entity_embeddings[i]
    entity['embedding_dim'] = len(entity_embeddings[i])

print(f"✅ Embeddings ajoutés à {len(all_entities)} entités")

### Génération des embeddings pour les chunks

In [ ]:
# Préparer les textes des chunks pour l'embedding
chunk_texts = [chunk_data['text'] for chunk_data in sample_chunks]

print(f"📄 Préparation de {len(chunk_texts)} chunks pour embedding")
print(f"Taille moyenne des chunks: {sum(len(text) for text in chunk_texts) // len(chunk_texts)} caractères\n")

# Générer les embeddings des chunks
chunk_embeddings = generate_embeddings(chunk_texts)

# Créer une structure de données enrichie pour les chunks
enriched_chunks = []
for i, chunk_data in enumerate(sample_chunks):
    enriched_chunk = {
        'id': chunk_data['id'],
        'text': chunk_data['text'],
        'embedding': chunk_embeddings[i],
        'embedding_dim': len(chunk_embeddings[i]),
        'filename': chunk_data['filename'],
        'doc_id': chunk_data['doc_id'],
        'chunk_index': chunk_data['chunk_index'],
        'text_length': len(chunk_data['text'])
    }
    enriched_chunks.append(enriched_chunk)

print(f"✅ Embeddings générés pour {len(enriched_chunks)} chunks")

## Création du Knowledge Graph dans Neo4j

Maintenant nous allons créer notre knowledge graph directement dans Neo4j en utilisant des requêtes Cypher.

### 1. Création des nœuds Chunk

In [ ]:
# Nettoyer la base avant création
print("🧹 Nettoyage de la base Neo4j...")
kg.query("MATCH (n) DETACH DELETE n")

# Créer les nœuds Chunk avec le texte
print("📄 Création des nœuds Chunk...")

for i, chunk in enumerate(enriched_chunks):
    chunk_query = """
    CREATE (c:Chunk {
        id: $chunk_id,
        text: $text,
        filename: $filename,
        doc_id: $doc_id,
        text_length: $text_length,
        chunk_index: $chunk_index
    })
    """
    
    kg.query(chunk_query, params={
        "chunk_id": chunk['id'],
        "text": chunk['text'],
        "filename": chunk['filename'],
        "doc_id": chunk['doc_id'],
        "text_length": chunk['text_length'],
        "chunk_index": chunk['chunk_index']
    })

print(f"✅ {len(enriched_chunks)} nœuds Chunk créés")

# Vérifier la création
result = kg.query("MATCH (c:Chunk) RETURN count(c) as chunk_count")
print(f"📊 Nœuds Chunk dans Neo4j: {result[0]['chunk_count']}")

### 2. Création des nœuds Entity

In [ ]:
# Créer les nœuds Entity
print("🏷️ Création des nœuds Entity...")

for entity in all_entities:
    entity_query = """
    CREATE (e:Entity {
        name: $name,
        type: $type,
        description: $description,
        confidence: $confidence,
        chunk_id: $chunk_id,
        source_chunk: $source_chunk
    })
    """
    
    kg.query(entity_query, params={
        "name": entity['name'],
        "type": entity['type'],
        "description": entity.get('description', ''),
        "confidence": entity.get('confidence', 0.8),
        "chunk_id": entity.get('chunk_id', ''),
        "source_chunk": entity.get('source_chunk', 0)
    })

print(f"✅ {len(all_entities)} nœuds Entity créés")

# Vérifier la création
result = kg.query("MATCH (e:Entity) RETURN count(e) as entity_count")
print(f"📊 Nœuds Entity dans Neo4j: {result[0]['entity_count']}")

# Voir la distribution par type
type_result = kg.query("""
    MATCH (e:Entity) 
    RETURN e.type as type, count(e) as count 
    ORDER BY count DESC
""")
print("\n📈 Distribution des entités par type:")
for row in type_result:
    print(f"   {row['type']}: {row['count']}")

### 3. Génération des embeddings avec genai.vector.encode

In [ ]:
# Générer les embeddings pour les chunks directement dans Neo4j
print("🔄 Génération des embeddings pour les chunks...")

# Vérifier d'abord si genai.vector.encode est disponible
try:
    test_query = "RETURN genai.vector.encode('test', 'OpenAI', {token: 'dummy'}) as test"
    kg.query(test_query)
    genai_available = True
    print("✅ Utilisation de genai.vector.encode pour les embeddings")
except:
    genai_available = False
    print("⚠️ genai.vector.encode non disponible - utilisation d'embeddings pre-calculés")

if genai_available:
    # Utiliser genai.vector.encode si disponible
    chunk_embedding_query = """
        MATCH (chunk:Chunk) WHERE chunk.textEmbedding IS NULL
        WITH chunk, genai.vector.encode(
          chunk.text, 
          "OpenAI", 
          {
            token: $openAiApiKey, 
            endpoint: $openAiEndpoint
          }) AS vector
        CALL db.create.setNodeVectorProperty(chunk, "textEmbedding", vector)
        RETURN count(chunk) as chunks_processed
    """
    
    result = kg.query(chunk_embedding_query, params={
        "openAiApiKey": OPENAI_API_KEY, 
        "openAiEndpoint": OPENAI_ENDPOINT
    })
    
    print(f"✅ Embeddings Neo4j générés pour {result[0]['chunks_processed']} chunks")
    
else:
    # Utiliser les embeddings pré-calculés
    print("🔄 Utilisation des embeddings pré-calculés...")
    
    for chunk in enriched_chunks:
        if 'embedding' in chunk:
            embedding_update_query = """
            MATCH (c:Chunk {id: $chunk_id})
            CALL db.create.setNodeVectorProperty(c, "textEmbedding", $embedding)
            RETURN c.id as updated_id
            """
            
            kg.query(embedding_update_query, params={
                "chunk_id": chunk['id'],
                "embedding": chunk['embedding']
            })
    
    print(f"✅ Embeddings pré-calculés ajoutés pour {len(enriched_chunks)} chunks")

# Même logique pour les entités
print("🔄 Génération des embeddings pour les entités...")

if genai_available:
    entity_embedding_query = """
        MATCH (entity:Entity) WHERE entity.textEmbedding IS NULL
        WITH entity, entity.name + " (" + entity.type + "): " + coalesce(entity.description, "") as entityText
        WITH entity, genai.vector.encode(
          entityText, 
          "OpenAI", 
          {
            token: $openAiApiKey, 
            endpoint: $openAiEndpoint
          }) AS vector
        CALL db.create.setNodeVectorProperty(entity, "textEmbedding", vector)
        RETURN count(entity) as entities_processed
    """
    
    result = kg.query(entity_embedding_query, params={
        "openAiApiKey": OPENAI_API_KEY, 
        "openAiEndpoint": OPENAI_ENDPOINT
    })
    
    print(f"✅ Embeddings Neo4j générés pour {result[0]['entities_processed']} entités")
    
else:
    print("🔄 Utilisation des embeddings pré-calculés pour les entités...")
    
    for entity in all_entities:
        if 'embedding' in entity:
            entity_embedding_update_query = """
            MATCH (e:Entity {name: $name, type: $type})
            CALL db.create.setNodeVectorProperty(e, "textEmbedding", $embedding)
            RETURN e.name as updated_name
            """
            
            kg.query(entity_embedding_update_query, params={
                "name": entity['name'],
                "type": entity['type'],
                "embedding": entity['embedding']
            })
    
    print(f"✅ Embeddings pré-calculés ajoutés pour {len(all_entities)} entités")

# Vérifier les embeddings
verification_query = """
    MATCH (n) 
    WHERE n.textEmbedding IS NOT NULL 
    RETURN labels(n)[0] as nodeType, count(n) as count, 
           size(n.textEmbedding) as embeddingDim
"""

verification_result = kg.query(verification_query)
print("\n📊 Vérification des embeddings:")
for row in verification_result:
    print(f"   {row['nodeType']}: {row['count']} nœuds, dimension {row['embeddingDim']}")

### 4. Création des relations MENTIONS

In [ ]:
# Créer les relations MENTIONS entre Chunk et Entity
print("🔗 Création des relations MENTIONS...")

mentions_created = 0

for entity in all_entities:
    # Créer la relation entre le chunk source et l'entité
    mentions_query = """
    MATCH (c:Chunk {id: $chunk_id})
    MATCH (e:Entity {name: $entity_name, type: $entity_type})
    CREATE (c)-[:MENTIONS {confidence: $confidence}]->(e)
    """
    
    kg.query(mentions_query, params={
        "chunk_id": entity.get('chunk_id', ''),
        "entity_name": entity['name'],
        "entity_type": entity['type'],
        "confidence": entity.get('confidence', 0.8)
    })
    mentions_created += 1

print(f"✅ {mentions_created} relations MENTIONS créées")

# Créer des relations NEXT entre chunks consécutifs
print("🔗 Création des relations NEXT entre chunks...")

next_query = """
MATCH (c1:Chunk), (c2:Chunk)
WHERE c1.chunk_index + 1 = c2.chunk_index
CREATE (c1)-[:NEXT]->(c2)
"""

kg.query(next_query)

# Vérifier les relations
relations_result = kg.query("""
    MATCH ()-[r]->() 
    RETURN type(r) as relationType, count(r) as count
""")

print("\n📊 Relations créées:")
for row in relations_result:
    print(f"   {row['relationType']}: {row['count']}")

print(f"\n🎯 Knowledge Graph créé avec succès!")
print(f"   Nœuds Chunk: {len(enriched_chunks)}")
print(f"   Nœuds Entity: {len(all_entities)}")
print(f"   Relations MENTIONS: {mentions_created}")
print(f"   Relations NEXT: {len(enriched_chunks)-1}")

## 🔗 Relations Avancées Inspirées de L5

Nous allons maintenant créer des relations plus sophistiquées basées sur les exemples du Lab 5, adaptées à notre structure de données PDF.

In [327]:
# 📋 CRÉATION DES NŒUDS DOCUMENT
# Similaire aux nœuds Form dans L5, nous créons des nœuds Document pour représenter chaque PDF

print("📄 Création des nœuds Document pour chaque PDF...")

# Récupérer les informations uniques des documents
doc_info_query = """
MATCH (c:Chunk) 
WITH c.filename as filename, c.doc_id as doc_id
RETURN DISTINCT filename, doc_id
"""

doc_infos = kg.query(doc_info_query)
print(f"   📚 {len(doc_infos)} documents uniques trouvés")

# Créer un nœud Document pour chaque PDF
for doc_info in doc_infos:
    create_doc_query = """
    MERGE (d:Document {doc_id: $doc_id})
    ON CREATE SET 
        d.filename = $filename,
        d.source = $filename,
        d.type = 'PDF'
    """
    
    kg.query(create_doc_query, params={
        'doc_id': doc_info['doc_id'],
        'filename': doc_info['filename']
    })

# Vérifier la création
doc_count = kg.query("MATCH (d:Document) RETURN count(d) as count")[0]['count']
print(f"✅ {doc_count} nœuds Document créés")

# Afficher les documents créés
docs = kg.query("MATCH (d:Document) RETURN d.filename as filename, d.doc_id as doc_id")
for doc in docs:
    print(f"   📄 {doc['filename']} (ID: {doc['doc_id'][:8]}...)")

📄 Création des nœuds Document pour chaque PDF...
   📚 10 documents uniques trouvés
   📚 10 documents uniques trouvés
✅ 10 nœuds Document créés
   📄 rapport-activite-2024-fin.pdf (ID: cdaa8e23...)
   📄 HW_GRAG_KnowledgeGraph_Solution_Engineer.pdf (ID: adcba2cb...)
   📄 HW_GRAG_KnowledgeGraph_Solution_Engineer.pdf (ID: f65e7891...)
   📄 rapport-activite-2024-fin.pdf (ID: 042b1062...)
   📄 HW_GRAG_KnowledgeGraph_Solution_Engineer.pdf (ID: a8d0c1d8...)
   📄 rapport-activite-2024-fin.pdf (ID: 03c6a6be...)
   📄 HW_GRAG_KnowledgeGraph_Solution_Engineer.pdf (ID: 3fef7c6a...)
   📄 rapport-activite-2024-fin.pdf (ID: 2cbf6174...)
   📄 HW_GRAG_KnowledgeGraph_Solution_Engineer.pdf (ID: d0d54f67...)
   📄 rapport-activite-2024-fin.pdf (ID: c4fbb0ae...)
✅ 10 nœuds Document créés
   📄 rapport-activite-2024-fin.pdf (ID: cdaa8e23...)
   📄 HW_GRAG_KnowledgeGraph_Solution_Engineer.pdf (ID: adcba2cb...)
   📄 HW_GRAG_KnowledgeGraph_Solution_Engineer.pdf (ID: f65e7891...)
   📄 rapport-activite-2024-fin.pdf (I

In [328]:
# 🔗 RELATIONS PART_OF - Chunks appartiennent aux Documents
# Similaire aux relations entre Chunks et Form dans L5

print("🔗 Création des relations PART_OF entre Chunks et Documents...")

part_of_query = """
MATCH (c:Chunk), (d:Document)
WHERE c.doc_id = d.doc_id
MERGE (c)-[r:PART_OF]->(d)
RETURN count(r) as relations_created
"""

result = kg.query(part_of_query)
print(f"✅ {result[0]['relations_created']} relations PART_OF créées")

# Vérifier les relations
verification_query = """
MATCH (c:Chunk)-[:PART_OF]->(d:Document)
RETURN d.filename as document, count(c) as chunk_count
ORDER BY chunk_count DESC
"""

doc_chunks = kg.query(verification_query)
print(f"\n📊 Distribution des chunks par document:")
for doc in doc_chunks:
    print(f"   📄 {doc['document']}: {doc['chunk_count']} chunks")

🔗 Création des relations PART_OF entre Chunks et Documents...


✅ 1416 relations PART_OF créées

📊 Distribution des chunks par document:
   📄 rapport-activite-2024-fin.pdf: 1401 chunks
   📄 HW_GRAG_KnowledgeGraph_Solution_Engineer.pdf: 15 chunks

📊 Distribution des chunks par document:
   📄 rapport-activite-2024-fin.pdf: 1401 chunks
   📄 HW_GRAG_KnowledgeGraph_Solution_Engineer.pdf: 15 chunks


In [329]:
# 🏷️ RELATIONS FIRST_CHUNK - Premier chunk de chaque document
# Similaire aux relations SECTION dans L5

print("🏷️ Création des relations FIRST_CHUNK...")

first_chunk_query = """
MATCH (c:Chunk), (d:Document)
WHERE c.doc_id = d.doc_id 
    AND c.chunk_index = 0
MERGE (d)-[r:FIRST_CHUNK]->(c)
RETURN count(r) as relations_created
"""

result = kg.query(first_chunk_query)
print(f"✅ {result[0]['relations_created']} relations FIRST_CHUNK créées")

# Vérifier les premiers chunks
first_chunks = kg.query("""
MATCH (d:Document)-[:FIRST_CHUNK]->(c:Chunk)
RETURN d.filename as document, c.id as chunk_id, 
       substring(c.text, 0, 100) as preview
""")

print(f"\n📋 Premiers chunks de chaque document:")
for chunk in first_chunks:
    print(f"   📄 {chunk['document']}")
    print(f"      ID: {chunk['chunk_id']}")
    print(f"      Aperçu: {chunk['preview']}...")
    print()

🏷️ Création des relations FIRST_CHUNK...
✅ 10 relations FIRST_CHUNK créées
✅ 10 relations FIRST_CHUNK créées

📋 Premiers chunks de chaque document:
   📄 rapport-activite-2024-fin.pdf
      ID: bfb5ab14-dbfb-45e7-9d92-70f27ab7e52e
      Aperçu: RAPPORT 
D’ACTIVITÉ 2024 
Ministère de la Digitalisation...

   📄 HW_GRAG_KnowledgeGraph_Solution_Engineer.pdf
      ID: 57e622ff-cc6a-4331-9965-5252e45f3759
      Aperçu: Technical Homework Assignment:  
 
 
GraphRAG Knowledge Graph endpoint + Streamlit 
frontend 
Object...

   📄 HW_GRAG_KnowledgeGraph_Solution_Engineer.pdf
      ID: 238803c8-933e-419b-aa73-5a346e3c88be
      Aperçu: Technical Homework Assignment:  
 
 
GraphRAG Knowledge Graph endpoint + Streamlit 
frontend 
Object...

   📄 rapport-activite-2024-fin.pdf
      ID: 1dd59428-04a6-4262-bb49-56f240a0971b
      Aperçu: RAPPORT 
D’ACTIVITÉ 2024 
Ministère de la Digitalisation...

   📄 HW_GRAG_KnowledgeGraph_Solution_Engineer.pdf
      ID: 105aca99-ee63-4165-8ac1-762786cdc71b
      Ape

In [330]:
# 🔗 AMÉLIORATION DES RELATIONS NEXT AVEC APOC (si disponible)
# Utilisation d'apoc.nodes.link comme dans L5 pour créer des liens plus robustes

print("🔗 Amélioration des relations NEXT avec apoc.nodes.link...")

# Vérifier si APOC est disponible
try:
    test_apoc = kg.query("RETURN apoc.version() as version")
    apoc_available = True
    print(f"✅ APOC disponible: {test_apoc[0]['version']}")
except:
    apoc_available = False
    print("⚠️ APOC non disponible - utilisation de la méthode standard")

if apoc_available:
    # Utiliser apoc.nodes.link pour chaque document séparément
    print("🔄 Recréation des relations NEXT avec APOC...")
    
    # D'abord supprimer les anciennes relations NEXT
    kg.query("MATCH ()-[r:NEXT]->() DELETE r")
    
    # Créer des relations NEXT pour chaque document avec APOC
    documents = kg.query("MATCH (d:Document) RETURN d.doc_id as doc_id, d.filename as filename")
    
    for doc in documents:
        apoc_next_query = """
        MATCH (c:Chunk)
        WHERE c.doc_id = $doc_id
        WITH c ORDER BY c.chunk_index ASC
        WITH collect(c) as chunk_list
        CALL apoc.nodes.link(chunk_list, "NEXT", {avoidDuplicates: true})
        RETURN size(chunk_list) as chunks_linked
        """
        
        result = kg.query(apoc_next_query, params={'doc_id': doc['doc_id']})
        print(f"   📄 {doc['filename']}: {result[0]['chunks_linked']} chunks liés")
    
    print("✅ Relations NEXT améliorées avec APOC")
else:
    print("✅ Relations NEXT existantes conservées")

# Vérifier les relations NEXT
next_count = kg.query("MATCH ()-[r:NEXT]->() RETURN count(r) as count")[0]['count']
print(f"\n📊 Total relations NEXT: {next_count}")

🔗 Amélioration des relations NEXT avec apoc.nodes.link...
✅ APOC disponible: 2025.08.0
🔄 Recréation des relations NEXT avec APOC...
   📄 rapport-activite-2024-fin.pdf: 17 chunks liés
   📄 HW_GRAG_KnowledgeGraph_Solution_Engineer.pdf: 3 chunks liés
   📄 rapport-activite-2024-fin.pdf: 17 chunks liés
   📄 HW_GRAG_KnowledgeGraph_Solution_Engineer.pdf: 3 chunks liés
   📄 HW_GRAG_KnowledgeGraph_Solution_Engineer.pdf: 3 chunks liés
   📄 rapport-activite-2024-fin.pdf: 346 chunks liés
   📄 HW_GRAG_KnowledgeGraph_Solution_Engineer.pdf: 3 chunks liés
   📄 rapport-activite-2024-fin.pdf: 346 chunks liés
   📄 HW_GRAG_KnowledgeGraph_Solution_Engineer.pdf: 3 chunks liés
   📄 rapport-activite-2024-fin.pdf: 346 chunks liés
   📄 HW_GRAG_KnowledgeGraph_Solution_Engineer.pdf: 3 chunks liés
   📄 rapport-activite-2024-fin.pdf: 346 chunks liés
   📄 HW_GRAG_KnowledgeGraph_Solution_Engineer.pdf: 3 chunks liés
   📄 rapport-activite-2024-fin.pdf: 346 chunks liés
   📄 HW_GRAG_KnowledgeGraph_Solution_Engineer.pdf: 

In [331]:
# 🤝 RELATIONS ENTRE ENTITÉS - Co-occurrence et Similarité
# Créer des relations entre entités qui apparaissent dans le même chunk ou des chunks similaires

print("🤝 Création de relations entre entités...")

# 1. Relations CO_OCCURS - Entités qui apparaissent dans le même chunk
co_occurrence_query = """
MATCH (e1:Entity)<-[:MENTIONS]-(c:Chunk)-[:MENTIONS]->(e2:Entity)
WHERE e1.name < e2.name  // Éviter les doublons
MERGE (e1)-[r:CO_OCCURS]->(e2)
ON CREATE SET r.shared_chunks = 1
ON MATCH SET r.shared_chunks = r.shared_chunks + 1
RETURN count(r) as co_occurrence_relations
"""

co_occurs = kg.query(co_occurrence_query)
print(f"✅ {co_occurs[0]['co_occurrence_relations']} relations CO_OCCURS créées")

# 2. Relations SAME_TYPE - Entités du même type
same_type_query = """
MATCH (e1:Entity), (e2:Entity)
WHERE e1.type = e2.type 
    AND e1.name < e2.name  // Éviter les doublons
    AND e1.type IN ['PERSON', 'ORGANIZATION', 'LOCATION']  // Limiter aux types importants
MERGE (e1)-[r:SAME_TYPE {type: e1.type}]->(e2)
RETURN count(r) as same_type_relations
"""

same_type = kg.query(same_type_query)
print(f"✅ {same_type[0]['same_type_relations']} relations SAME_TYPE créées")

# 3. Relations RELATED_TO - Entités liées par proximité dans le texte
proximity_query = """
MATCH (c1:Chunk)-[:NEXT]->(c2:Chunk)
MATCH (c1)-[:MENTIONS]->(e1:Entity)
MATCH (c2)-[:MENTIONS]->(e2:Entity)
WHERE e1.name <> e2.name
MERGE (e1)-[r:RELATED_TO]->(e2)
ON CREATE SET r.proximity_count = 1
ON MATCH SET r.proximity_count = r.proximity_count + 1
RETURN count(r) as proximity_relations
"""

proximity = kg.query(proximity_query)
print(f"✅ {proximity[0]['proximity_relations']} relations RELATED_TO créées")

print(f"\n📊 Résumé des relations entre entités:")
print(f"   🤝 Co-occurrence: {co_occurs[0]['co_occurrence_relations']}")
print(f"   🏷️ Même type: {same_type[0]['same_type_relations']}")
print(f"   🔗 Proximité: {proximity[0]['proximity_relations']}")

🤝 Création de relations entre entités...
✅ 447 relations CO_OCCURS créées
✅ 850 relations SAME_TYPE créées
✅ 447 relations CO_OCCURS créées
✅ 850 relations SAME_TYPE créées
✅ 862 relations RELATED_TO créées

📊 Résumé des relations entre entités:
   🤝 Co-occurrence: 447
   🏷️ Même type: 850
   🔗 Proximité: 862
✅ 862 relations RELATED_TO créées

📊 Résumé des relations entre entités:
   🤝 Co-occurrence: 447
   🏷️ Même type: 850
   🔗 Proximité: 862


In [332]:
# 📊 REQUÊTES D'EXEMPLE INSPIRÉES DE L5
# Démonstration des nouvelles relations avec des requêtes Cypher avancées

print("📊 EXEMPLES DE REQUÊTES AVANCÉES")
print("=" * 50)

# 1. Fenêtre de chunks autour d'un chunk spécifique (comme dans L5)
print("\n1️⃣ Fenêtre de chunks (contexte étendu):")

window_query = """
MATCH (c:Chunk)
WHERE c.text CONTAINS 'Luxembourg'
WITH c LIMIT 1
MATCH window = (prev:Chunk)-[:NEXT*0..1]->(c)-[:NEXT*0..1]->(next:Chunk)
WITH nodes(window) as chunk_window, c
UNWIND chunk_window as chunk_node
RETURN chunk_node.chunk_index as index, 
       substring(chunk_node.text, 0, 100) as text_preview
ORDER BY index
"""

window_result = kg.query(window_query)
if window_result:
    print("   📄 Contexte étendu autour du mot 'Luxembourg':")
    for chunk in window_result:
        print(f"   [{chunk['index']}] {chunk['text_preview']}...")
else:
    print("   ⚠️ Aucun chunk contenant 'Luxembourg' trouvé")

# 2. Navigation dans la structure de document
print("\n2️⃣ Navigation document -> chunks -> entités:")

navigation_query = """
MATCH (d:Document)-[:FIRST_CHUNK]->(first:Chunk)
MATCH (first)-[:NEXT*0..2]->(c:Chunk)-[:MENTIONS]->(e:Entity)
WHERE d.filename CONTAINS '.pdf'
RETURN d.filename as document, 
       c.chunk_index as chunk_index,
       e.name as entity, 
       e.type as entity_type
ORDER BY document, chunk_index
LIMIT 10
"""

navigation_result = kg.query(navigation_query)
print("   🗂️ Structure: Document -> Chunks -> Entités")
current_doc = None
for item in navigation_result:
    if item['document'] != current_doc:
        current_doc = item['document']
        print(f"   📄 {current_doc}")
    print(f"      Chunk[{item['chunk_index']}] -> {item['entity']} ({item['entity_type']})")

# 3. Entités co-occurrentes les plus fréquentes
print("\n3️⃣ Entités qui apparaissent souvent ensemble:")

co_occurrence_stats = """
MATCH (e1:Entity)-[r:CO_OCCURS]->(e2:Entity)
WHERE r.shared_chunks > 1
RETURN e1.name as entity1, e1.type as type1,
       e2.name as entity2, e2.type as type2,
       r.shared_chunks as frequency
ORDER BY frequency DESC
LIMIT 5
"""

co_occurrence_result = kg.query(co_occurrence_stats)
if co_occurrence_result:
    print("   🤝 Paires d'entités les plus liées:")
    for pair in co_occurrence_result:
        print(f"   {pair['entity1']} ({pair['type1']}) ↔ {pair['entity2']} ({pair['type2']}) [{pair['frequency']}x]")
else:
    print("   ⚠️ Aucune co-occurrence multiple trouvée")

print("\n" + "=" * 50)
print("✅ Exemples de requêtes exécutés avec succès!")

📊 EXEMPLES DE REQUÊTES AVANCÉES

1️⃣ Fenêtre de chunks (contexte étendu):
   📄 Contexte étendu autour du mot 'Luxembourg':
   [23] Rapport d’activité 2024 
Ministère de la Digitalisation  
 
8...
   [23] Rapport d’activité 2024 
Ministère de la Digitalisation  
 
8...
   [24] Rapport d’activité 2024 
Ministère de la Digitalisation  
 
9 
Trois stratégies nationales : données...
   [24] Rapport d’activité 2024 
Ministère de la Digitalisation  
 
9 
Trois stratégies nationales : données...
   [24] Rapport d’activité 2024 
Ministère de la Digitalisation  
 
9 
Trois stratégies nationales : données...
   [24] Rapport d’activité 2024 
Ministère de la Digitalisation  
 
9 
Trois stratégies nationales : données...
   [25] basée sur des projets innovants, la valorisation des données et l'application responsable et avantag...
   [25] basée sur des projets innovants, la valorisation des données et l'application responsable et avantag...

2️⃣ Navigation document -> chunks -> entités:
   🗂️ Struct

In [333]:
# 🚀 REQUÊTE DE RÉCUPÉRATION AVANCÉE AVEC FENÊTRE
# Implémentation d'une requête de récupération avec fenêtre de contexte comme dans L5

print("🚀 CONFIGURATION DE LA RÉCUPÉRATION AVEC FENÊTRE DE CONTEXTE")
print("=" * 60)

# Requête de récupération avec fenêtre inspirée de L5
retrieval_query_with_window = """
MATCH window = (prev:Chunk)-[:NEXT*0..1]->(node)-[:NEXT*0..1]->(next:Chunk)
WHERE node.doc_id = prev.doc_id OR prev IS NULL
  AND node.doc_id = next.doc_id OR next IS NULL
WITH node, score, window as contextWindow 
  ORDER BY length(window) DESC LIMIT 1
WITH nodes(contextWindow) as chunkList, node, score
  UNWIND chunkList as chunkNode
WITH collect(chunkNode.text) as textList, node, score
RETURN apoc.text.join(textList, " \\n\\n--- CHUNK SEPARATOR ---\\n\\n ") as text,
    score,
    {source: node.filename, chunk_id: node.id, window_size: size(textList)} AS metadata
"""

print("📋 Requête de récupération avec fenêtre créée!")
print("💡 Cette requête récupère le contexte étendu autour de chaque chunk trouvé")
print("💡 Similaire aux fenêtres de récupération du Lab 5")

# Fonction de test pour la récupération avec fenêtre
def test_window_retrieval():
    """Test de la récupération avec fenêtre de contexte"""
    print(f"\n🧪 TEST DE LA RÉCUPÉRATION AVEC FENÊTRE")
    print("-" * 40)
    
    # Test avec une requête simple
    test_query = """
    MATCH (c:Chunk)
    WHERE c.text CONTAINS 'Luxembourg'
    WITH c, 1.0 as score
    LIMIT 1
    
    MATCH window = (prev:Chunk)-[:NEXT*0..1]->(c)-[:NEXT*0..1]->(next:Chunk)
    WHERE (prev IS NULL OR prev.doc_id = c.doc_id)
      AND (next IS NULL OR next.doc_id = c.doc_id)
    WITH c, score, window as contextWindow 
      ORDER BY length(window) DESC LIMIT 1
    WITH nodes(contextWindow) as chunkList, c, score
    RETURN size(chunkList) as window_size,
           c.chunk_index as center_chunk,
           c.filename as source
    """
    
    try:
        result = kg.query(test_query)
        if result:
            window_info = result[0]
            print(f"✅ Test réussi!")
            print(f"   📊 Taille de fenêtre: {window_info['window_size']} chunks")
            print(f"   🎯 Chunk central: index {window_info['center_chunk']}")
            print(f"   📄 Source: {window_info['source']}")
        else:
            print("⚠️ Aucun résultat trouvé pour le test")
    except Exception as e:
        print(f"❌ Erreur dans le test: {str(e)[:100]}...")
        print("💡 La requête sera adaptée selon les capacités disponibles")

# Exécuter le test
test_window_retrieval()

print(f"\n✅ Configuration de récupération avec fenêtre prête!")
print(f"💡 Peut être utilisée avec Neo4jVector.from_existing_index()")
print(f"💡 Améliore le contexte des réponses RAG")

🚀 CONFIGURATION DE LA RÉCUPÉRATION AVEC FENÊTRE DE CONTEXTE
📋 Requête de récupération avec fenêtre créée!
💡 Cette requête récupère le contexte étendu autour de chaque chunk trouvé
💡 Similaire aux fenêtres de récupération du Lab 5

🧪 TEST DE LA RÉCUPÉRATION AVEC FENÊTRE
----------------------------------------
✅ Test réussi!
   📊 Taille de fenêtre: 3 chunks
   🎯 Chunk central: index 24
   📄 Source: rapport-activite-2024-fin.pdf

✅ Configuration de récupération avec fenêtre prête!
💡 Peut être utilisée avec Neo4jVector.from_existing_index()
💡 Améliore le contexte des réponses RAG
✅ Test réussi!
   📊 Taille de fenêtre: 3 chunks
   🎯 Chunk central: index 24
   📄 Source: rapport-activite-2024-fin.pdf

✅ Configuration de récupération avec fenêtre prête!
💡 Peut être utilisée avec Neo4jVector.from_existing_index()
💡 Améliore le contexte des réponses RAG


In [334]:
# 🎯 RÉCAPITULATIF FINAL DU KNOWLEDGE GRAPH ENRICHI
print("🎯 RÉCAPITULATIF FINAL DU KNOWLEDGE GRAPH ENRICHI")
print("=" * 60)

# Mettre à jour le schéma
kg.refresh_schema()

# Statistiques des nœuds
node_stats = kg.query("""
MATCH (n) 
RETURN labels(n)[0] as nodeType, count(n) as count
ORDER BY count DESC
""")

print("📊 NŒUDS DANS LE KNOWLEDGE GRAPH:")
total_nodes = 0
for stat in node_stats:
    count = stat['count']
    total_nodes += count
    print(f"   {stat['nodeType']:12} : {count:,} nœuds")

print(f"   {'TOTAL':12} : {total_nodes:,} nœuds")

# Statistiques des relations
relation_stats = kg.query("""
MATCH ()-[r]->() 
RETURN type(r) as relationType, count(r) as count
ORDER BY count DESC
""")

print(f"\n🔗 RELATIONS DANS LE KNOWLEDGE GRAPH:")
total_relations = 0
for stat in relation_stats:
    count = stat['count']
    total_relations += count
    print(f"   {stat['relationType']:12} : {count:,} relations")

print(f"   {'TOTAL':12} : {total_relations:,} relations")

# Capacités ajoutées depuis L5
print(f"\n🚀 NOUVELLES CAPACITÉS AJOUTÉES (inspirées de L5):")
capabilities = [
    "📄 Nœuds Document pour structurer les PDFs",
    "🔗 Relations PART_OF (chunks → documents)",
    "🏷️ Relations FIRST_CHUNK (navigation dans documents)",
    "🔄 Relations NEXT améliorées avec APOC",
    "🤝 Relations CO_OCCURS entre entités co-occurrentes",
    "🏷️ Relations SAME_TYPE entre entités similaires",
    "📍 Relations RELATED_TO basées sur la proximité",
    "🪟 Requêtes de fenêtre de contexte pour RAG",
    "🎯 Navigation avancée dans la structure du graphe"
]

for capability in capabilities:
    print(f"   ✅ {capability}")

# Exemples de requêtes avancées disponibles
print(f"\n🔍 EXEMPLES DE REQUÊTES AVANCÉES DISPONIBLES:")
queries = [
    "Fenêtre de contexte autour d'un chunk",
    "Navigation Document → Chunk → Entités", 
    "Entités co-occurrentes fréquentes",
    "Chemins entre entités liées",
    "Récupération RAG avec contexte étendu"
]

for query in queries:
    print(f"   🎯 {query}")

print(f"\n🌟 KNOWLEDGE GRAPH PRÊT POUR UNE UTILISATION AVANCÉE!")
print(f"   • Structure hiérarchique: Document → Chunk → Entity")
print(f"   • Relations sémantiques entre entités")
print(f"   • Navigation contextuelle avec fenêtres")
print(f"   • Compatible avec les techniques avancées de RAG")

print(f"\n📈 SCHÉMA DU GRAPHE:")
print(kg.schema)

🎯 RÉCAPITULATIF FINAL DU KNOWLEDGE GRAPH ENRICHI
📊 NŒUDS DANS LE KNOWLEDGE GRAPH:
   Chunk        : 1,416 nœuds
   Entity       : 108 nœuds
   Document     : 10 nœuds
   TOTAL        : 1,534 nœuds

🔗 RELATIONS DANS LE KNOWLEDGE GRAPH:
   PART_OF      : 1,416 relations
   NEXT         : 1,406 relations
   RELATED_TO   : 854 relations
   SAME_TYPE    : 850 relations
   CO_OCCURS    : 423 relations
   MENTIONS     : 144 relations
   FIRST_CHUNK  : 10 relations
   TOTAL        : 5,103 relations

🚀 NOUVELLES CAPACITÉS AJOUTÉES (inspirées de L5):
   ✅ 📄 Nœuds Document pour structurer les PDFs
   ✅ 🔗 Relations PART_OF (chunks → documents)
   ✅ 🏷️ Relations FIRST_CHUNK (navigation dans documents)
   ✅ 🔄 Relations NEXT améliorées avec APOC
   ✅ 🤝 Relations CO_OCCURS entre entités co-occurrentes
   ✅ 🏷️ Relations SAME_TYPE entre entités similaires
   ✅ 📍 Relations RELATED_TO basées sur la proximité
   ✅ 🪟 Requêtes de fenêtre de contexte pour RAG
   ✅ 🎯 Navigation avancée dans la structure du gra

## Visualisation du Knowledge Graph

Maintenant, visualisons notre Knowledge Graph directement dans le notebook avec plusieurs méthodes.

In [335]:
# Fonction pour récupérer les données du graphe
def get_graph_data_from_neo4j():
    """Récupère les données du graphe Neo4j pour visualisation"""
    
    # Récupérer tous les nœuds
    nodes_query = """
    MATCH (n) 
    RETURN id(n) as node_id, labels(n)[0] as label, 
           n.name as name, n.type as type, n.id as chunk_id
    """
    nodes_data = kg.query(nodes_query)
    
    # Récupérer toutes les relations
    edges_query = """
    MATCH (a)-[r]->(b) 
    RETURN id(a) as source, id(b) as target, type(r) as relation_type,
           r.confidence as confidence
    """
    edges_data = kg.query(edges_query)
    
    return nodes_data, edges_data

# Récupérer les données
nodes_data, edges_data = get_graph_data_from_neo4j()

print(f"📊 Données récupérées:")
print(f"   Nœuds: {len(nodes_data)}")
print(f"   Relations: {len(edges_data)}")

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function. ('id' has been replaced by 'elementId or consider using an application-generated id')} {position: line: 3, column: 12, offset: 27} for query: '\n    MATCH (n) \n    RETURN id(n) as node_id, labels(n)[0] as label, \n           n.name as name, n.type as type, n.id as chunk_id\n    '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function. ('id' has been replaced by 'elementId or consider using an application-generated id')} {position: line: 3, column: 12, offset: 36} for query: '\n    MATCH (a)-[

📊 Données récupérées:
   Nœuds: 1534
   Relations: 5103


## Visualisation avancée avec yFiles Jupyter Graphs pour Neo4j

yFiles pour Neo4j offre une intégration directe avec la base de données pour des visualisations très avancées avec des layouts automatiques et une interactivité poussée.

### Création du widget yFiles Neo4j avec requête Cypher directe

In [336]:
driver = GraphDatabase.driver(
    uri = NEO4J_URI, 
    auth = (NEO4J_USERNAME, NEO4J_PASSWORD), 
    database = NEO4J_DATABASE
    )
g = Neo4jGraphWidget(driver)



### Visualisation du Knowledge Graph avec requête Cypher

In [337]:
# Afficher le Knowledge Graph avec une requête Cypher simple
# Exactement comme dans votre exemple
g.show_cypher("MATCH (s)-[r]->(t) RETURN s,r,t")

print("🎉 Knowledge Graph affiché!")
print("🔍 Vous pouvez interagir avec le graphe:")
print("   • Zoom avec la molette")
print("   • Déplacer les nœuds")
print("   • Cliquer pour sélectionner")
print("   • Layouts automatiques disponibles")

GraphWidget(layout=Layout(height='800px', width='100%'))

🎉 Knowledge Graph affiché!
🔍 Vous pouvez interagir avec le graphe:
   • Zoom avec la molette
   • Déplacer les nœuds
   • Cliquer pour sélectionner
   • Layouts automatiques disponibles


In [338]:
# 🎉 PIPELINE KNOWLEDGE GRAPH COMPLET !
print("🎉 FÉLICITATIONS ! Votre Knowledge Graph est opérationnel")
print("=" * 60)

print("✅ ÉTAPES ACCOMPLIES:")
print("   1. 📄 Extraction et chunking du PDF")
print("   2. 🤖 Extraction d'entités avec OpenAI GPT-3.5")
print("   3. 🔗 Génération d'embeddings OpenAI")
print("   4. 🗄️ Création du Knowledge Graph dans Neo4j")
print("   5. 📊 Visualisation avec yFiles Neo4j (connexion directe)")

# Statistiques finales
with driver.session() as session:
    chunk_count = session.run("MATCH (c:Chunk) RETURN count(c) as count").single()["count"]
    entity_count = session.run("MATCH (e:Entity) RETURN count(e) as count").single()["count"]
    mentions_count = session.run("MATCH ()-[r:MENTIONS]->() RETURN count(r) as count").single()["count"]

print(f"\n📊 STATISTIQUES DU KNOWLEDGE GRAPH:")
print(f"   📚 Chunks de texte: {chunk_count}")
print(f"   🏷️ Entités extraites: {entity_count}")
print(f"   🔗 Relations MENTIONS: {mentions_count}")

print(f"\n🛠️ OUTILS UTILISÉS:")
print(f"   • Neo4j (base de données graphe)")
print(f"   • OpenAI GPT-3.5-turbo (extraction d'entités)")
print(f"   • OpenAI text-embedding-3-small (embeddings)")
print(f"   • yFiles Jupyter Graphs pour Neo4j (visualisation)")
print(f"   • LangChain (orchestration)")

print(f"\n🎯 UTILISATION:")
print(f"   • Exécutez: g.show_cypher('VOTRE_REQUETE_CYPHER')")
print(f"   • Explorez les entités par type")
print(f"   • Analysez les relations sémantiques")
print(f"   • Utilisez les layouts automatiques de yFiles")

print(f"\n🚀 PROCHAINES ÉTAPES SUGGÉRÉES:")
print(f"   1. Ajoutez d'autres documents pour enrichir le graphe")
print(f"   2. Implémentez des requêtes de recherche sémantique")
print(f"   3. Créez des vues personnalisées avec Cypher")
print(f"   4. Exploitez les embeddings pour la similarité")

print(f"\n🎨 VISUALISATION INTERACTIVE PRÊTE!")
print(f"Utilisez les cellules ci-dessus pour explorer votre Knowledge Graph avec yFiles.")

# Message de fin
print(f"\n" + "🌟" * 20)
print(f"   KNOWLEDGE GRAPH RAG OPÉRATIONNEL")
print(f"🌟" * 20)

🎉 FÉLICITATIONS ! Votre Knowledge Graph est opérationnel
✅ ÉTAPES ACCOMPLIES:
   1. 📄 Extraction et chunking du PDF
   2. 🤖 Extraction d'entités avec OpenAI GPT-3.5
   3. 🔗 Génération d'embeddings OpenAI
   4. 🗄️ Création du Knowledge Graph dans Neo4j
   5. 📊 Visualisation avec yFiles Neo4j (connexion directe)

📊 STATISTIQUES DU KNOWLEDGE GRAPH:
   📚 Chunks de texte: 1416
   🏷️ Entités extraites: 108
   🔗 Relations MENTIONS: 144

🛠️ OUTILS UTILISÉS:
   • Neo4j (base de données graphe)
   • OpenAI GPT-3.5-turbo (extraction d'entités)
   • OpenAI text-embedding-3-small (embeddings)
   • yFiles Jupyter Graphs pour Neo4j (visualisation)
   • LangChain (orchestration)

🎯 UTILISATION:
   • Exécutez: g.show_cypher('VOTRE_REQUETE_CYPHER')
   • Explorez les entités par type
   • Analysez les relations sémantiques
   • Utilisez les layouts automatiques de yFiles

🚀 PROCHAINES ÉTAPES SUGGÉRÉES:
   1. Ajoutez d'autres documents pour enrichir le graphe
   2. Implémentez des requêtes de recherche sé

In [339]:
# 🔧 CONFIGURATION CORRIGÉE POUR NEO4J VECTOR STORE
# Utiliser le bon nom d'index qui existe vraiment dans votre base
CORRECT_VECTOR_INDEX_NAME = 'GrahRAG'  # Le vrai nom de votre index

print(f"🔧 Configuration Neo4j Vector Store:")
print(f"   Index: {CORRECT_VECTOR_INDEX_NAME}")
print(f"   Node Label: {VECTOR_NODE_LABEL}")
print(f"   Text Property: {VECTOR_SOURCE_PROPERTY}")
print(f"   Embedding Property: {VECTOR_EMBEDDING_PROPERTY}")

try:
    neo4j_vector_store = Neo4jVector.from_existing_graph(
        embedding=OpenAIEmbeddings(),
        url=NEO4J_URI,
        username=NEO4J_USERNAME,
        password=NEO4J_PASSWORD,
        index_name=CORRECT_VECTOR_INDEX_NAME,  # Utiliser le bon nom
        node_label=VECTOR_NODE_LABEL,
        text_node_properties=[VECTOR_SOURCE_PROPERTY],
        embedding_node_property=VECTOR_EMBEDDING_PROPERTY,
    )
    print("✅ Neo4j Vector Store configuré avec succès")
    
except Exception as e:
    print(f"❌ Erreur configuration Neo4j Vector Store: {str(e)[:200]}...")
    print("💡 Vérification des paramètres:")
    
    # Vérifier l'index
    indexes = kg.query("SHOW INDEXES")
    print(f"   Indexes disponibles: {[idx['name'] for idx in indexes]}")
    
    # Vérifier les nœuds
    nodes = kg.query("MATCH (n:Chunk) RETURN count(n) as count")
    print(f"   Nœuds Chunk: {nodes[0]['count'] if nodes else 0}")
    
    neo4j_vector_store = None

🔧 Configuration Neo4j Vector Store:
   Index: GrahRAG
   Node Label: Chunk
   Text Property: text
   Embedding Property: textEmbedding
✅ Neo4j Vector Store configuré avec succès
✅ Neo4j Vector Store configuré avec succès


In [340]:
# 🔧 CRÉATION DU RETRIEVER AVEC GESTION D'ERREUR
if neo4j_vector_store is not None:
    try:
        retriever = neo4j_vector_store.as_retriever()
        print("✅ Retriever configuré avec succès")
    except Exception as e:
        print(f"❌ Erreur création retriever: {str(e)[:200]}...")
        retriever = None
else:
    print("⚠️ Vector store non disponible - retriever non créé")
    retriever = None

✅ Retriever configuré avec succès


In [341]:
# 🔧 CRÉATION DE LA CHAÎNE RAG AVEC GESTION D'ERREUR
if retriever is not None:
    try:
        chain = RetrievalQAWithSourcesChain.from_chain_type(
            ChatOpenAI(temperature=0), 
            chain_type="stuff", 
            retriever=retriever
        )
        print("✅ Chaîne RAG configurée avec succès")
        chain_available = True
    except Exception as e:
        print(f"❌ Erreur création chaîne RAG: {str(e)[:200]}...")
        chain = None
        chain_available = False
else:
    print("⚠️ Retriever non disponible - chaîne RAG non créée")
    chain = None
    chain_available = False

✅ Chaîne RAG configurée avec succès


In [342]:
# 🤖 FONCTION RAG AMÉLIORÉE AVEC FALLBACK
def prettychain(question: str) -> str:
    """Fonction RAG robuste avec fallback sur recherche vectorielle"""
    
    print(f"🤔 Question: {question}")
    print("-" * 50)
    
    # Méthode 1: Essayer la chaîne RAG standard
    if chain_available and chain is not None:
        try:
            print("🔄 Utilisation de la chaîne RAG standard...")
            response = chain({"question": question}, return_only_outputs=True)
            answer = response.get('answer', 'Pas de réponse')
            sources = response.get('sources', 'Pas de sources')
            
            print("✅ Réponse RAG:")
            print(textwrap.fill(answer, 80))
            print(f"\n📚 Sources: {sources}")
            return answer
            
        except Exception as e:
            print(f"❌ Erreur chaîne RAG: {str(e)[:100]}...")
            print("🔄 Basculement vers recherche vectorielle...")
    
    # Méthode 2: Fallback avec recherche vectorielle directe
    try:
        print("🔍 Utilisation de la recherche vectorielle directe...")
        results = neo4j_vector_search_robust(question, top_k=3)
        
        if results:
            # Construire une réponse à partir des résultats
            context_parts = []
            sources = []
            
            for i, result in enumerate(results[:3]):
                context_parts.append(result['text'])
                sources.append(result['source'])
            
            context = "\n\n".join(context_parts)
            unique_sources = list(set(sources))
            
            # Utiliser OpenAI pour générer une réponse basée sur le contexte
            from langchain_openai import ChatOpenAI
            llm = ChatOpenAI(temperature=0)
            
            prompt = f"""Basé sur le contexte suivant, réponds à la question: "{question}"
            
Contexte:
{context}

Réponds de manière concise et factuelle en français."""
            
            ai_response = llm.invoke(prompt)
            answer = ai_response.content
            
            print("✅ Réponse basée sur recherche vectorielle:")
            print(textwrap.fill(answer, 80))
            print(f"\n📚 Sources: {', '.join(unique_sources)}")
            return answer
            
        else:
            print("❌ Aucun résultat trouvé dans la recherche vectorielle")
            return "Désolé, je n'ai pas trouvé d'informations pertinentes pour cette question."
            
    except Exception as e:
        print(f"❌ Erreur recherche vectorielle: {str(e)[:100]}...")
        return "Erreur lors de la recherche d'informations."

# 🧪 FONCTION DE TEST SIMPLE
def test_rag_system():
    """Test du système RAG"""
    
    print("🧪 TEST DU SYSTÈME RAG")
    print("=" * 40)
    
    test_questions = [
        "c'est quoi Luxchatgov",
        "Luxembourg",
        "What technologies are mentioned?"
    ]
    
    for question in test_questions:
        print(f"\n🔍 Test: {question}")
        result = prettychain(question)
        print("=" * 40)

print("✅ Fonctions RAG améliorées créées!")
print("💡 Utilisez prettychain('votre question') pour tester")
print("💡 Utilisez test_rag_system() pour un test complet")

✅ Fonctions RAG améliorées créées!
💡 Utilisez prettychain('votre question') pour tester
💡 Utilisez test_rag_system() pour un test complet


In [343]:
prettychain("c'est quoi Luxchatgov")

🤔 Question: c'est quoi Luxchatgov
--------------------------------------------------
🔄 Utilisation de la chaîne RAG standard...
❌ Erreur chaîne RAG: Document prompt requires documents to have metadata variables: ['source']. Received document with mi...
🔄 Basculement vers recherche vectorielle...
🔍 Utilisation de la recherche vectorielle directe...
🔍 Recherche pour: 'c'est quoi Luxchatgov'
❌ Erreur chaîne RAG: Document prompt requires documents to have metadata variables: ['source']. Received document with mi...
🔄 Basculement vers recherche vectorielle...
🔍 Utilisation de la recherche vectorielle directe...
🔍 Recherche pour: 'c'est quoi Luxchatgov'
✅ Recherche réussie: 3 résultats trouvés
   1. Score: 0.7545 | Source: rapport-activite-2024-fin.pdf
      Text: Appel à projets Tech-in-GOV ...........................................................................
   2. Score: 0.7370 | Source: rapport-activite-2024-fin.pdf
      Text: Luxchat4Gov ...........................................

"Luxchat4Gov est un projet lié à la digitalisation de l'administration publique au Luxembourg."

In [344]:
# 🛠️ FONCTION PRETTYCHAIN ROBUSTE - CORRIGÉE
def prettychain(question: str) -> str:
    """Fonction RAG robuste avec gestion des métadonnées source manquantes"""
    
    print(f"🤔 Question: {question}")
    print("-" * 50)
    
    # D'abord essayer la recherche vectorielle directe qui fonctionne
    try:
        print("🔍 Utilisation de la recherche vectorielle directe...")
        results = neo4j_vector_search_robust(question, top_k=3)
        
        if results:
            # Construire une réponse à partir des résultats
            context_parts = []
            sources = []
            
            for i, result in enumerate(results[:3]):
                context_parts.append(result['text'])
                sources.append(result['source'])
            
            context = "\n\n".join(context_parts)
            unique_sources = list(set(sources))
            
            # Utiliser OpenAI pour générer une réponse basée sur le contexte
            from langchain_openai import ChatOpenAI
            llm = ChatOpenAI(temperature=0)
            
            prompt = f"""Basé sur le contexte suivant, réponds à la question: "{question}"
            
Contexte:
{context}

Réponds de manière concise et factuelle en français."""
            
            ai_response = llm.invoke(prompt)
            answer = ai_response.content
            
            print("✅ Réponse basée sur recherche vectorielle:")
            print(textwrap.fill(answer, 80))
            print(f"\n📚 Sources: {', '.join(unique_sources)}")
            return answer
            
        else:
            print("❌ Aucun résultat trouvé dans la recherche vectorielle")
            return "Désolé, je n'ai pas trouvé d'informations pertinentes pour cette question."
            
    except Exception as e:
        print(f"❌ Erreur recherche vectorielle: {str(e)[:100]}...")
        
        # Fallback: essayer la chaîne RAG avec correction des métadonnées
        try:
            print("🔄 Tentative avec chaîne RAG corrigée...")
            
            # Récupérer les documents via le retriever
            docs = retriever.get_relevant_documents(question)
            
            # Corriger les métadonnées manquantes
            for doc in docs:
                if 'source' not in doc.metadata:
                    # Utiliser le nom de fichier si disponible, sinon valeur par défaut
                    if hasattr(doc, 'page_content') and len(doc.page_content) > 0:
                        doc.metadata['source'] = "Knowledge Graph"
                    else:
                        doc.metadata['source'] = "Unknown"
            
            # Utiliser la chaîne avec les documents corrigés
            response = chain({"question": question}, return_only_outputs=True)
            answer = response.get('answer', 'Pas de réponse')
            sources = response.get('sources', 'Pas de sources')
            
            print("✅ Réponse RAG corrigée:")
            print(textwrap.fill(answer, 80))
            print(f"\n📚 Sources: {sources}")
            return answer
            
        except Exception as e2:
            print(f"❌ Erreur chaîne RAG: {str(e2)[:100]}...")
            return "Erreur lors de la recherche d'informations."

print("✅ Fonction prettychain robuste créée!")
print("💡 Cette version gère les métadonnées source manquantes")
print("💡 Elle utilise la recherche vectorielle comme méthode principale")

✅ Fonction prettychain robuste créée!
💡 Cette version gère les métadonnées source manquantes
💡 Elle utilise la recherche vectorielle comme méthode principale


In [345]:
# 🧪 TEST DE LA FONCTION PRETTYCHAIN CORRIGÉE
print("🧪 Test de la fonction prettychain corrigée:")
print("=" * 50)

# Test avec la même question qui causait l'erreur
test_result = prettychain("c'est quoi Luxchatgov")

🧪 Test de la fonction prettychain corrigée:
🤔 Question: c'est quoi Luxchatgov
--------------------------------------------------
🔍 Utilisation de la recherche vectorielle directe...
🔍 Recherche pour: 'c'est quoi Luxchatgov'
✅ Recherche réussie: 3 résultats trouvés
   1. Score: 0.7544 | Source: rapport-activite-2024-fin.pdf
      Text: Appel à projets Tech-in-GOV ...........................................................................
   2. Score: 0.7370 | Source: rapport-activite-2024-fin.pdf
      Text: Luxchat4Gov ...........................................................................................
   3. Score: 0.7319 | Source: rapport-activite-2024-fin.pdf
      Text: MyGuichet.lu ..........................................................................................
✅ Recherche réussie: 3 résultats trouvés
   1. Score: 0.7544 | Source: rapport-activite-2024-fin.pdf
      Text: Appel à projets Tech-in-GOV ..................................................................